# PyTorch

In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

## Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1) # 3은 RGB channel로 입력됨을 의미함
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Preprocess

In [4]:
train_dir = '../dataset/mnist_png/training/'
test_dir = '../dataset/mnist_png/testing/'

grayscale은 안 되는 이유

https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py#L157

In [5]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


train_dataset = datasets.ImageFolder(root=train_dir,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))

test_dataset = datasets.ImageFolder(root=test_dir,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))

train_loader = torch.utils.data.DataLoader(train_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size=test_batch_size
                                         )

## Optimization

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

## Training

In [7]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()  # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # pred와 target과 같은지 확인

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301064
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.243285
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.127935
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.844007
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.390902
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.886382
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.576883
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.522230
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.520606
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.529002

Test set: Average loss: 0.3751, Accuracy: 8935/10000 (89%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.436779
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.359864
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.333890
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.283140
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.399463
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.207738
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.183059
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.436433
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.249445
T